Librerias a usarse


In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
import numpy as np
import re
import pandas as pd

Se extraen las diferentes hojas de nuestra busqueda en MercadoLibre, es importante pasar a la segunda hoja y regresar a ala primera, pues cambia la direccion web al hacer la busqueda y al regresar a la primer pagina.
Los items se agrupan de 50 en 50, en caso de querer explorar mas paginas necesitamos agregarle 50 

In [ ]:
page = requests.get("https://celulares.mercadolibre.com.mx/samsung_NoIndex_True")
page2 = requests.get("https://celulares.mercadolibre.com.mx/samsung_Desde_51_NoIndex_True")
page3 = requests.get("https://celulares.mercadolibre.com.mx/samsung_Desde_101_NoIndex_True")

Pare verificar que la conexion se hizo adecuadamente imprimimos el requests, este nos tiene que devolver la respuesta 200

In [ ]:
print(page)
print(page2)
print(page3)

<Response [200]>
<Response [200]>
<Response [200]>


Se inidica que se quiere extraer el texto de un html

In [ ]:
soup = BeautifulSoup(page.text,"html")
soup2 = BeautifulSoup(page2.text,"html")
soup3 = BeautifulSoup(page3.text,"html")

In [ ]:
soup

<!DOCTYPE html>
<html lang="es-MX">
<head><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://www.google.com" rel="preconnect"/><link href="https://data.mercadolibre.com" rel="preconnect"/><link href="https://http2.mlstatic.com" rel="preconnect"/><link href="https://stats.g.doubleclick.net" rel="preconnect"/><link href="https://analytics.mercadolibre.com.mx" rel="preconnect"/><link href="https://analytics.mercadolibre.com" rel="preconnect"/><link href="https://www.google.com.mx" rel="preconnect"/><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-689ffbd95eae88e39ac","applicationID":"1729522169","applicationTime":1084.153221,"transactionName":"YlZQYEVZC0QEV0BZV1scd0xHSgBEFl5HH39wZx0bHQ==","queueTime":0,"ttGuid":"8eda8a838dd18bed","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]}

Se hace la comprobacion de una pagina

In [ ]:
for i in soup.find_all("div",attrs={'class':'ui-search-result__content-wrapper'}):
  print(i)

<div class="ui-search-result__content-wrapper"><div class="ui-search-item__highlight-label ui-search-item__highlight-label--deal_of_the_day" style="background:#3483FA"><div class="ui-search-item__highlight-label__container"><label class="ui-search-styled-label ui-search-item__highlight-label__text" for="" style="color:#FFFFFF">OFERTA DEL DÍA</label></div></div><div class="ui-search-item__group ui-search-item__group--title"><span class="ui-search-item__brand-discoverability ui-search-item__group__element"></span><a class="ui-search-item__group__element ui-search-link" href="https://www.mercadolibre.com.mx/samsung-galaxy-a52s-5g-128-gb-awesome-black-6-gb-ram/p/MLM18577244?pdp_filters=category:MLM1055#searchVariation=MLM18577244&amp;position=1&amp;search_layout=stack&amp;type=product&amp;tracking_id=56062814-3b06-4cbd-a6cb-9f4a36e45bc3" title="Samsung Galaxy A52s 5G 128 GB awesome black 6 GB RAM"><h2 class="ui-search-item__title">Samsung Galaxy A52s 5G 128 GB awesome black 6 GB RAM</h2></

A continuacion se extrae la infomracion que queremos y la vamos ir añadiendo a una lista para posterioremente crear un dataframe

In [ ]:
#Creacion de listas
celular1 = []
precio1 = []
vendedor1 = [] 
#Bucle para la primera pagina
for i in soup.find_all("div",attrs={'class':'ui-search-result__content-wrapper'}): #se extrae uno de los divs mostrados anteriormente
  title = i.find('h2',attrs={'class':"ui-search-item__title"}) #Se hace la busqueda de la etiquetab y con la clase nos tare el elemento especifico
  celular1.append(title.text) # se añade a la lista y es importante indicar el text, de lo contrario te tarera toda la etiqueta
  price = i.find('span',attrs={"class" : "price-tag-fraction"})
  precio1.append(price.text)
  vendedor = i.find('p',attrs={"class" : "ui-search-official-store-label ui-search-item__group__element ui-search-color--GRAY"})
  if vendedor:  # Es imnportante este paso, hay articulos sin vendeodres y esto nos puede cuasar un error
    vendedor1.append(vendedor.text)
  else:vendedor1.append("SIN VENDEDOR")

#Bucle para la segunda pagina
for i in soup2.find_all("div",attrs={'class':'ui-search-result__content-wrapper'}):
  title = i.find('h2',attrs={'class':"ui-search-item__title"})
  celular1.append(title.text)
  price = i.find('span',attrs={"class" : "price-tag-fraction"})
  precio1.append(price.text)
  vendedor = i.find('p',attrs={"class" : "ui-search-official-store-label ui-search-item__group__element ui-search-color--GRAY"})
  if vendedor: 
    vendedor1.append(vendedor.text)
  else:vendedor1.append("SIN VENDEDOR")

#Bucle para la tercera pagina
for i in soup3.find_all("div",attrs={'class':'ui-search-result__content-wrapper'}):
  title = i.find('h2',attrs={'class':"ui-search-item__title"})
  celular1.append(title.text)
  price = i.find('span',attrs={"class" : "price-tag-fraction"})
  precio1.append(price.text)
  vendedor = i.find('p',attrs={"class" : "ui-search-official-store-label ui-search-item__group__element ui-search-color--GRAY"})
  if vendedor: 
    vendedor1.append(vendedor.text)
  else:vendedor1.append("SIN VENDEDOR")

In [ ]:
len(celular1)

168

In [ ]:
len(precio1)

168

In [ ]:
len(vendedor1)

168

In [ ]:
# Se crea dataframe con los elementos extraidos en los bucles 
ML = pd.DataFrame(list(zip(celular1, precio1,vendedor1)),columns =['Celular', 'Precio',"Vendedor"])

In [ ]:
#Visualizamos la informacion
ML

,Celular,Precio,Vendedor
0,Samsung Galaxy A52s 5G 128 GB awesome black 6 ...,"10,499",Vendido por Mercado Libre Electronica
1,Samsung Galaxy A53 5G 128 GB awesome black 6 G...,"6,985",Vendido por Telmov
2,Samsung Galaxy A52 128 GB azul sorprendente 6 ...,"5,499",Vendido por Compupaz
3,Samsung Galaxy A52 Dual SIM 128 GB violeta sor...,"5,293",SIN VENDEDOR
4,Samsung Galaxy A32 128 GB awesome black 4 GB RAM,"4,799",Vendido por Celmishop
...,...,...,...
163,Celular Samsung Galaxy A23 Azul 128gb 4gb Ram,"4,899",Vendido por Telmov
164,Samsung Galaxy S10+ 128 Gb Blanco Prisma 8 Gb ...,"9,997",Vendido por MX WH
165,Samsung Galaxy S10 128gb 8gb Ram Azul Liberado...,"8,497",Vendido por MX WH
166,Samsung Galaxy A13 128gb - 4gb Ram Desbloquead...,"4,099",Vendido por GC Movil
